# Projet 5A ENR

## Apprendre la stratégie journalière de stockage
### Mise en place d'un réseau de neurone

### Objectifs : 
- Estimer le coût d’un mix énergétique de manière rapide en limitant le temps de calcul de l'optimisation du stockage qui se fait aujourd’hui heure par heure. 

L’entrée du réseau peut être par exemple :

1) L’état des stocks des différents moyens de stockage (en valeur absolue ou en pourcentage) à 0h

2) La production nette de la journée (production totale de la journée - consommation totale)

3) le jour de l’année (entre 1 et 365)

Et les sorties seraient : l’état des stocks en fin journée des différents moyens de stockages et le cout de stockage de la journée.
Ceci n’st qu’une proposition. On peut aussi mettre en entrée, la variance de la production journalière qui peut avoir un rôle.

In [28]:
import matplotlib.pyplot as plt
from matplotlib import ticker
import numpy as np
import pandas as pd
import seaborn as sns
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

## Prétraitement des données

- il faut genérer les données avec les modèles 

- Quel mix choisir ? Le choisir et l'importer ici =) 

- Le cout de stockage n'est t'il pas aussi dépendant du profil de consommation ? (estimation ADEME, Negawhatt, RTE ?) réponse : le profil de consommation intervient dans la production de la journée nette (qui n'est pas un partie logique)

- Comment on fait pour connaître la production de la journée idem pour la consommation ? Ducoup c'est online ? Quelle idée derrière la variance de la production journalière (+ de variance plus de stockage ?) ? 
- C'est pas un peu con de connaître exactement la production de la journée ? parceque ducoup le stock prédis ca va être sensiblement stock d'entree + prod de la journée : un peu étrange selon Flavie


- produire les données d'apprentissage contenant

    - 1 jour de l'année entre 1 et 365
    - état des stocks à h0
    - production de la journée
    - variance production journalière.
    - variance consomation journalière utile ? (peut être bien que oui aussi)





In [29]:
simu_opti_ademe = pd.read_csv("Project_Renewables_2050_Stage_Amandine/simulations/optimal_simulations/simu_opti_ademe/simu_opti_ademe_hourly_generation.csv")

In [30]:
simu_opti_ademe.drop(["Electricity demand","hour","reserve phs","reserve phs","reserve battery","reserve lake","Storage phs","Storage battery","Storage methanation"],inplace = True, axis =1)
#simu_opti_ademe.drop("hour",inplace = True, axis =1)

In [31]:
simu_opti_ademe

,offshore,onshore,pv,river,lake,biogas,phs,battery,methanation,Stored phs,Stored battery,Stored methanation
0,10.94,40.80,0.0,1.27,0.0,0.0,0.0,0.0,0.0,0.00,0.0,8143.11
1,11.12,42.09,0.0,1.30,0.0,0.0,0.0,0.0,0.0,0.00,0.0,8147.63
2,11.12,42.89,0.0,1.11,0.0,0.0,0.0,0.0,0.0,0.00,0.0,8152.15
3,11.04,43.18,0.0,1.12,0.0,0.0,0.0,0.0,0.0,0.00,0.0,8156.67
4,10.77,42.95,0.0,1.21,0.0,0.0,0.0,0.0,0.0,6.01,0.0,8161.19
...,...,...,...,...,...,...,...,...,...,...,...,...
8755,12.37,65.05,0.0,2.79,0.0,0.0,0.0,0.0,0.0,0.00,0.0,8120.52
8756,12.47,65.34,0.0,2.72,0.0,0.0,0.0,0.0,0.0,0.00,0.0,8125.04
8757,12.52,65.87,0.0,2.45,0.0,0.0,0.0,0.0,0.0,0.00,0.0,8129.56
8758,12.53,65.79,0.0,2.24,0.0,0.0,0.0,0.0,0.0,0.00,0.0,8134.08


In [32]:
#transforming Data from hour to day
#sum over generation
#hour 24 for stored

In [33]:
A =simu_opti_ademe[["offshore","onshore","pv","river","lake","biogas","phs","battery","methanation"]].groupby(np.arange(len(simu_opti_ademe))//24).sum()
A

,offshore,onshore,pv,river,lake,biogas,phs,battery,methanation
0,248.21,987.76,167.14,32.00,0.00,0.00,0.00,0.00,0.00
1,89.07,520.50,270.72,37.68,0.00,0.00,12.19,0.00,220.56
2,55.50,323.21,310.81,45.47,0.00,0.00,62.90,1.22,630.18
3,162.95,530.71,272.38,68.40,0.00,0.00,105.15,56.84,289.52
4,142.35,486.05,239.21,76.71,65.00,45.05,2.96,12.38,395.69
...,...,...,...,...,...,...,...,...,...
360,81.91,229.73,373.09,35.84,312.00,0.00,42.05,57.71,342.09
361,140.42,365.48,389.76,41.46,312.00,0.00,147.90,71.17,82.72
362,261.86,872.92,314.12,43.67,136.91,0.00,20.44,7.40,0.00
363,288.11,1405.12,235.34,46.40,0.00,0.00,0.00,0.00,0.00


In [34]:
B = simu_opti_ademe[["Stored phs","Stored battery","Stored methanation"]][::24]
B.reset_index(inplace = True)

In [35]:
B

,index,Stored phs,Stored battery,Stored methanation
0,0,0.00,0.00,8143.11
1,24,163.12,74.14,8251.58
2,48,180.00,74.14,7824.72
3,72,110.11,72.86,6424.34
4,96,1.96,13.04,5781.00
...,...,...,...,...
360,8640,136.08,41.32,8792.97
361,8664,133.30,17.25,8032.79
362,8688,22.71,7.79,7848.96
363,8712,0.00,0.00,7926.18


In [36]:
Inputs =[A,B]
Inputs =pd.concat(Inputs,axis =1)

In [37]:
Inputs

,offshore,onshore,pv,river,lake,biogas,phs,battery,methanation,index,Stored phs,Stored battery,Stored methanation
0,248.21,987.76,167.14,32.00,0.00,0.00,0.00,0.00,0.00,0,0.00,0.00,8143.11
1,89.07,520.50,270.72,37.68,0.00,0.00,12.19,0.00,220.56,24,163.12,74.14,8251.58
2,55.50,323.21,310.81,45.47,0.00,0.00,62.90,1.22,630.18,48,180.00,74.14,7824.72
3,162.95,530.71,272.38,68.40,0.00,0.00,105.15,56.84,289.52,72,110.11,72.86,6424.34
4,142.35,486.05,239.21,76.71,65.00,45.05,2.96,12.38,395.69,96,1.96,13.04,5781.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...
360,81.91,229.73,373.09,35.84,312.00,0.00,42.05,57.71,342.09,8640,136.08,41.32,8792.97
361,140.42,365.48,389.76,41.46,312.00,0.00,147.90,71.17,82.72,8664,133.30,17.25,8032.79
362,261.86,872.92,314.12,43.67,136.91,0.00,20.44,7.40,0.00,8688,22.71,7.79,7848.96
363,288.11,1405.12,235.34,46.40,0.00,0.00,0.00,0.00,0.00,8712,0.00,0.00,7926.18


In [38]:
Inputs["days"]= Inputs.index.values +1
Inputs

,offshore,onshore,pv,river,lake,biogas,phs,battery,methanation,index,Stored phs,Stored battery,Stored methanation,days
0,248.21,987.76,167.14,32.00,0.00,0.00,0.00,0.00,0.00,0,0.00,0.00,8143.11,1
1,89.07,520.50,270.72,37.68,0.00,0.00,12.19,0.00,220.56,24,163.12,74.14,8251.58,2
2,55.50,323.21,310.81,45.47,0.00,0.00,62.90,1.22,630.18,48,180.00,74.14,7824.72,3
3,162.95,530.71,272.38,68.40,0.00,0.00,105.15,56.84,289.52,72,110.11,72.86,6424.34,4
4,142.35,486.05,239.21,76.71,65.00,45.05,2.96,12.38,395.69,96,1.96,13.04,5781.00,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
360,81.91,229.73,373.09,35.84,312.00,0.00,42.05,57.71,342.09,8640,136.08,41.32,8792.97,361
361,140.42,365.48,389.76,41.46,312.00,0.00,147.90,71.17,82.72,8664,133.30,17.25,8032.79,362
362,261.86,872.92,314.12,43.67,136.91,0.00,20.44,7.40,0.00,8688,22.71,7.79,7848.96,363
363,288.11,1405.12,235.34,46.40,0.00,0.00,0.00,0.00,0.00,8712,0.00,0.00,7926.18,364


In [39]:
# ajouter le coût du storage
# overnight cost $/kW
# CAPEX $/kWh
# Annuity $/kW/year > pas lié au stockage
# Fixed O&M $/MWh
# Storage annuity $/kWh/year

In [40]:
stor = ["phs","battery","methanation"]
tec = ["offshore","onshore","pv","river","lake","biogas","phs","battery","methanation"]

# Annualized power capex cost in M€/GW/year
capex = pd.read_csv("inputs/annuities.csv", index_col=0, squeeze=True, header=None)
# Existing capacities of the technologies by December 2017 in GW
capa_ex = pd.read_csv("inputs/existing_capas.csv", index_col=0, squeeze=True, header=None)
# Annualized energy capex cost of storage technologies in M€/GWh/year
capex_en = pd.read_csv("inputs/str_annuities.csv", index_col=0, squeeze=True, header=None)
# Annualized fixed operation and maintenance costs M€/GW/year
fOM = pd.read_csv("inputs/fO&M.csv", index_col=0, squeeze=True, header=None)
# Variable operation and maintenance costs in M€/GWh
vOM = pd.read_csv("inputs/vO&M.csv", index_col=0, squeeze=True, header=None)



# variables optimisés par les 5A 2021
s_capex = [0,0,84.16086]
s_capex = pd.Series(s_capex, index = stor)
# charging related fOM of storage in M€/GW/year
s_opex = [0,0,59.25]
s_opex = pd.Series(s_opex, index = stor)



/tmp/ipykernel_2575/950090184.py:5: FutureWarning: The squeeze argument has been deprecated and will be removed in a future version. Append .squeeze("columns") to the call to squeeze.


  capex = pd.read_csv("inputs/annuities.csv", index_col=0, squeeze=True, header=None)
/tmp/ipykernel_2575/950090184.py:7: FutureWarning: The squeeze argument has been deprecated and will be removed in a future version. Append .squeeze("columns") to the call to squeeze.


  capa_ex = pd.read_csv("inputs/existing_capas.csv", index_col=0, squeeze=True, header=None)
/tmp/ipykernel_2575/950090184.py:9: FutureWarning: The squeeze argument has been deprecated and will be removed in a future version. Append .squeeze("columns") to the call to squeeze.


  capex_en = pd.read_csv("inputs/str_annuities.csv", index_col=0, squeeze=True, header=None)
/tmp/ipykernel_2575/950090184.py:11: FutureWarning: The squeeze argument has been deprecated and will be removed in a future version. Append .squeeze("columns") to the ca

In [41]:
vOM = vOM.rename(index={"Onshore": "onshore", "PV":"pv", "PHS":"phs", "Battery":"battery"})
fOM = fOM.rename(index={"Offshore": "offshore", "Onshore": "onshore", "PV":"pv", "PHS":"phs", "Battery":"battery"})
capex = capex.rename(index={"Offshore": "offshore", "Onshore": "onshore", "PV":"pv", "PHS":"phs", "Battery":"battery"})

In [42]:
tec = ["offshore","onshore","pv","river","lake","biogas","phs","battery","methanation"]
Q = pd.Series([12.64, 79.73, 121.98, 7.5, 13 ,32.93, 9.3, 20.08, 32.93], index = tec)
volume= pd.Series([180 ,74.14, 12499.09], index = stor)
S = pd.Series([9.3,20.08, 7.66], index = stor)
gene = Inputs[["phs","battery","methanation"]]

In [43]:
gene

,phs,battery,methanation
0,0.00,0.00,0.00
1,12.19,0.00,220.56
2,62.90,1.22,630.18
3,105.15,56.84,289.52
4,2.96,12.38,395.69
...,...,...,...
360,42.05,57.71,342.09
361,147.90,71.17,82.72
362,20.44,7.40,0.00
363,0.00,0.00,0.00


In [44]:
DAY_FIXED_COST = (sum((Q[tec] - capa_ex[tec]) * capex[tec] for tec in stor) \
+ sum(volume[storage_tecs] * capex_en[storage_tecs] for storage_tecs in stor)\
+ sum(Q[tec] * fOM[tec] for tec in stor)\
+ sum(S[storage_tecs] * (s_opex[storage_tecs] + s_capex[storage_tecs]) for storage_tecs in stor)\
)/(365*1000)

In [45]:
DAY_VAR_COST = sum((gene[tec] * vOM[tec]) for tec in stor)/1000

In [46]:
COST = DAY_VAR_COST +DAY_FIXED_COST

In [47]:
Inputs

,offshore,onshore,pv,river,lake,biogas,phs,battery,methanation,index,Stored phs,Stored battery,Stored methanation,days
0,248.21,987.76,167.14,32.00,0.00,0.00,0.00,0.00,0.00,0,0.00,0.00,8143.11,1
1,89.07,520.50,270.72,37.68,0.00,0.00,12.19,0.00,220.56,24,163.12,74.14,8251.58,2
2,55.50,323.21,310.81,45.47,0.00,0.00,62.90,1.22,630.18,48,180.00,74.14,7824.72,3
3,162.95,530.71,272.38,68.40,0.00,0.00,105.15,56.84,289.52,72,110.11,72.86,6424.34,4
4,142.35,486.05,239.21,76.71,65.00,45.05,2.96,12.38,395.69,96,1.96,13.04,5781.00,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
360,81.91,229.73,373.09,35.84,312.00,0.00,42.05,57.71,342.09,8640,136.08,41.32,8792.97,361
361,140.42,365.48,389.76,41.46,312.00,0.00,147.90,71.17,82.72,8664,133.30,17.25,8032.79,362
362,261.86,872.92,314.12,43.67,136.91,0.00,20.44,7.40,0.00,8688,22.71,7.79,7848.96,363
363,288.11,1405.12,235.34,46.40,0.00,0.00,0.00,0.00,0.00,8712,0.00,0.00,7926.18,364


In [48]:
#créer les Xtest, Ytest, Xtrain, Ytrain

In [49]:
X = Inputs[:-1]
X

,offshore,onshore,pv,river,lake,biogas,phs,battery,methanation,index,Stored phs,Stored battery,Stored methanation,days
0,248.21,987.76,167.14,32.00,0.00,0.00,0.00,0.00,0.00,0,0.00,0.00,8143.11,1
1,89.07,520.50,270.72,37.68,0.00,0.00,12.19,0.00,220.56,24,163.12,74.14,8251.58,2
2,55.50,323.21,310.81,45.47,0.00,0.00,62.90,1.22,630.18,48,180.00,74.14,7824.72,3
3,162.95,530.71,272.38,68.40,0.00,0.00,105.15,56.84,289.52,72,110.11,72.86,6424.34,4
4,142.35,486.05,239.21,76.71,65.00,45.05,2.96,12.38,395.69,96,1.96,13.04,5781.00,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
359,31.26,163.78,394.24,35.37,312.00,0.00,12.66,32.22,470.65,8616,111.29,61.23,9838.84,360
360,81.91,229.73,373.09,35.84,312.00,0.00,42.05,57.71,342.09,8640,136.08,41.32,8792.97,361
361,140.42,365.48,389.76,41.46,312.00,0.00,147.90,71.17,82.72,8664,133.30,17.25,8032.79,362
362,261.86,872.92,314.12,43.67,136.91,0.00,20.44,7.40,0.00,8688,22.71,7.79,7848.96,363


In [50]:
Y = Inputs[1:]
Y = Y[["Stored phs","Stored battery","Stored methanation"]]
Y["cost"]=COST[1:]
Y

,Stored phs,Stored battery,Stored methanation,cost
1,163.12,74.14,8251.58,0.007905
2,180.00,74.14,7824.72,0.010138
3,110.11,72.86,6424.34,0.008400
4,1.96,13.04,5781.00,0.008884
5,0.00,0.00,4901.66,0.009756
...,...,...,...,...
360,136.08,41.32,8792.97,0.008688
361,133.30,17.25,8032.79,0.007304
362,22.71,7.79,7848.96,0.006720
363,0.00,0.00,7926.18,0.006704


In [51]:
from sklearn.model_selection import train_test_split 
X_train,X_test,Y_train,Y_test=train_test_split(X,Y,test_size=0.25,random_state=11)

## Model de réseau de neurones

Il s'agit d'un truc au hasard d'internet à affiner 
d'internet : 

'Pour la couche Dropout, on a diminué de 30% le nombre des données d’entrée afin d’éviter le phénomène du overfitting. La graine prend une valeur de 2 pour avoir des résultats plus reproductibles.' from https://www.cours-gratuit.com/tutoriel-python/tutoriel-python-matriser-les-rseaux-de-neurones-avec-keras

In [52]:
model = keras.Sequential([
layers.Dense(64, activation = 'relu', input_shape = [X_train.shape[1]]),
layers.Dropout(0.3, seed = 2),
layers.Dense(64, activation = 'swish'),
layers.Dense(64, activation = 'relu'),
layers.Dense(64, activation = 'swish'),
layers.Dense(64, activation = 'relu'),
layers.Dense(64, activation = 'swish'),
layers.Dense(4)])

### Liste des optimizer à choisir

class Adadelta: Optimizer that implements the Adadelta algorithm.

class Adagrad: Optimizer that implements the Adagrad algorithm.

class Adam: Optimizer that implements the Adam algorithm.

class Adamax: Optimizer that implements the Adamax algorithm.

class Ftrl: Optimizer that implements the FTRL algorithm.

class Nadam: Optimizer that implements the NAdam algorithm.

class Optimizer: Base class for Keras optimizers.

class RMSprop: Optimizer that implements the RMSprop algorithm.

class SGD: Gradient descent (with momentum) optimizer.

## Liste des losses à choisir

class BinaryCrossentropy: Computes the cross-entropy loss between true labels and predicted labels.

class BinaryFocalCrossentropy: Computes the focal cross-entropy loss between true labels and predictions.

class CategoricalCrossentropy: Computes the crossentropy loss between the labels and predictions.

class CategoricalHinge: Computes the categorical hinge loss between y_true and y_pred.

class CosineSimilarity: Computes the cosine similarity between labels and predictions.

class Hinge: Computes the hinge loss between y_true and y_pred.

class Huber: Computes the Huber loss between y_true and y_pred.

class KLDivergence: Computes Kullback-Leibler divergence loss between y_true and y_pred.

class LogCosh: Computes the logarithm of the hyperbolic cosine of the prediction error.

class Loss: Loss base class.

class MeanAbsoluteError: Computes the mean of absolute difference between labels and predictions.

class MeanAbsolutePercentageError: Computes the mean absolute percentage error between y_true and y_pred.

class MeanSquaredError: Computes the mean of squares of errors between labels and predictions.

class MeanSquaredLogarithmicError: Computes the mean squared logarithmic error between y_true and y_pred.

class Poisson: Computes the Poisson loss between y_true and y_pred.

class Reduction: Types of loss reduction.

class SparseCategoricalCrossentropy: Computes the crossentropy loss between the labels and predictions.

class SquaredHinge: Computes the squared hinge loss between y_true and y_pred.

## Choisir une metrics

In [53]:
optimiseur = tf.keras.optimizers.RMSprop(learning_rate = 0.001)
model.compile(loss = tf.keras.losses.MeanSquaredError(),
optimizer = optimiseur,
metrics = ['accuracy'])

training = model.fit (X_train, Y_train, epochs = 70, validation_split = 0.2)

Epoch 1/70
7/7 [==============================] - 2s 99ms/step - loss: 10734305.0000 - accuracy: 0.6682 - val_loss: 1190990.5000 - val_accuracy: 0.9636
Epoch 2/70
7/7 [==============================] - 0s 16ms/step - loss: 668657.1719 - accuracy: 0.9453 - val_loss: 175548.1406 - val_accuracy: 0.9636
Epoch 3/70
7/7 [==============================] - 0s 15ms/step - loss: 282831.8789 - accuracy: 0.9491 - val_loss: 133126.7500 - val_accuracy: 0.9636
Epoch 4/70
7/7 [==============================] - 0s 19ms/step - loss: 271639.1328 - accuracy: 0.9632 - val_loss: 64718.9180 - val_accuracy: 0.9636
Epoch 5/70
7/7 [==============================] - 0s 27ms/step - loss: 301957.2070 - accuracy: 0.9662 - val_loss: 42262.6328 - val_accuracy: 0.9636
Epoch 6/70
7/7 [==============================] - 0s 23ms/step - loss: 312857.5742 - accuracy: 0.9698 - val_loss: 40844.8359 - val_accuracy: 0.9636
Epoch 7/70
7/7 [==============================] - 0s 17ms/step - loss: 212166.9824 - accuracy: 0.9557 - va

7/7 [==============================] - 0s 11ms/step - loss: 123238.1660 - accuracy: 0.9671 - val_loss: 766283.5000 - val_accuracy: 0.9636
Epoch 56/70
7/7 [==============================] - 0s 11ms/step - loss: 115841.3604 - accuracy: 0.9550 - val_loss: 1130331.6250 - val_accuracy: 0.9818
Epoch 57/70
7/7 [==============================] - 0s 11ms/step - loss: 77005.4863 - accuracy: 0.9680 - val_loss: 709230.3125 - val_accuracy: 0.9818
Epoch 58/70
7/7 [==============================] - 0s 13ms/step - loss: 93895.2236 - accuracy: 0.9721 - val_loss: 456808.9375 - val_accuracy: 0.9636
Epoch 59/70
7/7 [==============================] - 0s 15ms/step - loss: 199163.7793 - accuracy: 0.9742 - val_loss: 824285.0625 - val_accuracy: 0.9636
Epoch 60/70
7/7 [==============================] - 0s 12ms/step - loss: 90039.5781 - accuracy: 0.9560 - val_loss: 559341.8125 - val_accuracy: 0.9818
Epoch 61/70
7/7 [==============================] - 0s 12ms/step - loss: 122066.7051 - accuracy: 0.9565 - val_loss:

In [54]:
score =model.evaluate(X_test,Y_test)

3/3 [==============================] - 0s 8ms/step - loss: 1372447.6250 - accuracy: 0.9451
